<a href="https://colab.research.google.com/github/jayjinnie/gbdcon/blob/main/%EA%B2%BD%EB%B6%81_%EC%9E%A0%EC%9E%AC_%EC%9A%94%EC%9D%B8_%ED%98%91%EC%97%85_%ED%95%84%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 잠재 요인 협업 필터링 (Latent Factor Collaborative Filtering)
#### 행렬 분해(Matrix Factorization)를 기반으로 잠재 요인을 추출하여 추출된 잠재 요인을 기반으로 상품을 추천함

# 필요한 데이터
#### customer_id: 회원번호
#### merc_name: 상품명
#### rating: 선호도


In [1]:
import pandas as pd

data = pd.read_excel("선호도 테이블.xlsx")
data.head()

,회원번호,상품명,선호도
0,1,무농약 연근 쓱결제용,18.680282
1,1,★이웃사촌★ [성주로컬푸드]고소애500g,20.818027
2,1,★이웃사촌★ [성주로컬푸드]고소애500g,20.818027
3,1,[오세창곤충체험농장]파워뱅이분말 180g,18.806200
4,158993,[오세창곤충체험농장]파워뱅이분말 180g,18.806200


# pivot 테이블 만들기
#### 고객-상품 선호도 데이터 pivot 테이블 생성
#### fillna로 NaN 값 채워주기

In [ ]:
customer_merc_rating = data.pivot_table('선호도', index='회원번호', columns='상품명').fillna(0)
customer_merc_rating.shape

In [ ]:
customer_merc_rating.head()

# 특정 상품과 유사한 상품을 추천
#### 따라서 고객-상품 테이블을 상품-고객 테이블로 전치 (transpose)

In [ ]:
merc_customer_rating = customer_merc_rating.values.T
merc_customer_rating.shape

# SVD (Singular Value Decomposition) 
#### 특이값 분해를 통해 유사한 데이터, 잠재 요인 추출

In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD

# 5개의 주요 feature를 가진 행렬로 데이터 세트 변환
SVD = TruncatedSVD(n_components=5)
matrix = SVD.fit_transform(merc_customer_rating)
matrix.shape

In [ ]:
matrix[0]

#### 1261개의 상품 데이터가 5개의 어떠한 요소의 값을 가지게 됨
#### 추출된 데이터끼리의 피어슨 상관계수 구하기

In [ ]:
corr = np.corrcoef(matrix)
corr.shape

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12,10))
sns.heatmap(corr, cmap="PuBu", annot = False)

In [ ]:
# 상관계수를 이용하여 특정 상품과 상관계수가 높은 다른 상품 추출
merc_name = customer_merc_rating.columns
merc_name_list = list(merc_name)
temp_data = merc_name_list.index("피크닉사과 가정용 흠과 10kg")

In [ ]:
corr_temp_data = corr[temp_data]
list(merc_name[(corr_temp_data >= 0.9)])[:50]

### 추출한 잠재 요인을 기반으로 특정 상품과 유사한 상품을 추천해주는 시스템
### 그러나 고객의 구매 이력을 고려하지 않으므로 고객에게 맞춤형으로 추천하기에는 한계가 있음